# Title

In [50]:
from icalendar import Calendar, Event
from datetime import datetime
import time as Time
import uuid
import os

calendar = {}
prodid = "1" #TODO: change this to our name

def export(file):
	os.startfile(file)

def process_ics(content):
	try:
		input = content.split(",")
		date = input[0]
		time = input[1]
		summary = input[2]
		date = date.split("/")
		endtime=""
		if "全天" in time:
			time = "00:00~23:59"
		if "~" in time:
			time = time.split("~")
			endtime= time[1]
			time = time[0]
		calendar[summary] = (summary, "{:04d}".format(int(date[0]))+"-"+"{:02d}".format(int(date[1]))+"-"+"{:02d}".format(int(date[2])), time, endtime)
		time = time.split(":")
		if endtime!="":
			endtime = endtime.split(":")

		cal = Calendar()
		cal.add('version', '2.0')
		cal.add('prodid', prodid)

		event = Event()

		now_time = datetime(
		Time.localtime()[0],
		Time.localtime()[1],
		Time.localtime()[2],
		Time.localtime()[3],
		Time.localtime()[4],
		Time.localtime()[5])

		event.add('summary', summary)
		event.add('dtstart', datetime(int(date[0]), int(date[1]), int(date[2]), int(time[0]), int(time[1])))
		if endtime!="":
			event.add('dtend', datetime(int(date[0]), int(date[1]), int(date[2]), int(endtime[0]), int(endtime[1])))
		else:
			event.add('dtend', datetime(int(date[0]), int(date[1]), int(date[2]), int(time[0]), int(time[1])))
		event.add('dtstamp', now_time)
		uid = uuid.uuid1()
		event['uid'] = uid
		cal.add_component(event)

		f = open(f'{uid}.ics', 'wb')
		f.write(cal.to_ical())
		f.close()

		export(f'{uid}.ics')
	except:
		print(f"处理失败: {content}")

def to_ics(response):
	response=response.splitlines()
	for content in response:
		process_ics(content)

def getEvents():
	result = ""
	for summary, date, time, endtime in calendar:
		result = result + """			{
				__TITLE__,
				__START____END__
			}__COMMA__""".replace("__TITLE__", f"title: '{summary}'").replace("__START__", f"start: '{date}T{time}:00'").replace("__END__", f",\n\t\t\t\tend: '{date}T{endtime}:00'" if endtime!="" else "")
	result = result + "DISABLE__"
	return result.replace("__COMMA__DISABLE__", "").replace("__COMMA__", ",\n")

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

device = "cuda" if torch.cuda.is_available() else "cpu"

torch.cuda.empty_cache()

model = AutoModelForCausalLM.from_pretrained(
	"Qwen/Qwen2-7B-Instruct",
	torch_dtype="auto",
	device_map="balanced_low_0"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct")
streamer = TextStreamer(tokenizer, skip_prompt = True, skip_special_tokens = True)

c:\ProgramData\Anaconda3\Lib\site-packages\accelerate\utils\modeling.py:1405: UserWarning: Current model requires 385878912 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu and disk.


In [3]:
time = str(Time.localtime()[0])+'/'+str(Time.localtime()[1])+'/'+str(Time.localtime()[2])
messages = [
	{
		"role": "system", 
		"content": f"""今天的日期是 {time} ，现在请说中文。user会给你一些文字，请你从文字中提取信息，告诉user在这些文字中交代了在什么时间要做什么事情。以这样的格式回答：
	日期, 时间, 事件
	要求：日期格式为 YYYY/MM/DD ，时间格式为 HH:mm 。如果时间是 12 小时制，转换为 24 小时制。。如果没有提到时间，时间填写 全天 。仅当提供了开始时间和结束时间（或者可由时长计算）时，时间格式为 HH:mm ~ HH:mm，代表开始时间、结束时间。如果只提供了开始时间而没有提到结束时间，以 HH:mm 格式回答开始时间。如果没有交代任何事件，请不要回答任何内容。事件请为其概况内容，在总结时不要包含日期和时间。
	"""
	}
]

def process(prompt):
	messages.append({"role": "user", "content": prompt})
	text = tokenizer.apply_chat_template(
		messages,
		tokenize=False,
		add_generation_prompt=True
	)
	model_inputs = tokenizer([text], return_tensors="pt").to(device)

	generated_ids = model.generate(
		**model_inputs,
		max_new_tokens = 512,
		streamer = streamer
	)
	generated_ids = [
		output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
	]

	response = tokenizer.batch_decode(generated_ids, skip_special_tokens = True)[0]
	messages.append({"role": "assistant", "content": response})

	response = response.replace("，", ",")
	to_ics(response)

	return response

In [51]:
import gradio as gr

head = """<head>
	<meta charset='utf-8' />
	<script src='https://cdn.jsdelivr.net/npm/fullcalendar@6.1.15/index.global.min.js'></script>
</head>"""
js = """function loadCalendar() {
	var calendarEl = document.getElementById('calendar');
	var calendar = new FullCalendar.Calendar(calendarEl, {
		events: [
__EVENTS_REPLACE_TAG__
		]
	});
	calendar.render();
}
""".replace("__EVENTS_REPLACE_TAG__", getEvents())
print(js)
css = """.calendar {
    height: 90vh;
}

.fc-button {
	margin-bottom: auto !important;
}
"""

with gr.Blocks(head = head, js=js, css=css) as ui:
	gr.HTML("""<div id="calendar" class="calendar"></div>""")

ui.launch()

function loadCalendar() {
	var calendarEl = document.getElementById('calendar');
	var calendar = new FullCalendar.Calendar(calendarEl, {
		events: [
			{
				title: '睡觉',
				start: '2024-08-03T20:00:00',
				end: '2024-08-03T21:00:00'
			},
			{
				title: '吃饭',
				start: '2024-08-03T20:00:00'
			}
		]
	});
	calendar.render();
}

Running on local URL:  http://127.0.0.1:7887

To create a public link, set `share=True` in `launch()`.


## TODO List

语音

冲突检测
